In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Step 1: Load and Preprocess CIFAR-10 Dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize the images
x_train, x_test = x_train / 255.0, x_test / 255.0

# Convert labels to one-hot encoding
y_train, y_test = to_categorical(y_train, 10), to_categorical(y_test, 10)

# Create ImageDataGenerators for training and testing
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow(x_train, y_train, batch_size=32)
test_generator = test_datagen.flow(x_test, y_test, batch_size=32)

# Step 2: Load and Prepare the Pre-trained VGG16 Model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Add custom layers on top of the VGG16 base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)  # 10 classes for CIFAR-10

# Define the complete model
model = Model(inputs=base_model.input, outputs=predictions)

# Step 3: Freeze the Pre-trained Layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Step 4: Train the Model
history = model.fit(train_generator, epochs=5, validation_data=test_generator)

# Step 5: Unfreeze and Fine-Tune the Model
for layer in base_model.layers[-4:]:  # Unfreeze last 4 layers
    layer.trainable = True

# Recompile the model with a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tune the model
history_fine_tune = model.fit(train_generator, epochs=5, validation_data=test_generator)

# Step 6: Evaluate the Model
eval_results = model.evaluate(test_generator)
print('Test Loss, Test Accuracy:', eval_results)

# Save the model
model.save('fine_tuned_vgg16_cifar10.h5')

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1563/1563 ━━━━━━━━━━━━━━━━━━━━ 701s 448ms/step - accuracy: 0.4102 - loss: 1.6598 - val_accuracy: 0.5115 - val_loss: 1.3626
Epoch 2/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 701s 449ms/step - accuracy: 0.4976 - loss: 1.4173 - val_accuracy: 0.5337 - val_loss: 1.2924
Epoch 3/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 700s 448ms/step - accuracy: 0.5131 - loss: 1.3704 - val_accuracy: 0.5640 - val_loss: 1.2276
Epoch 4/5
1143/1563 ━━━━━━━━━━━━━━━━━━━━ 2:37 375ms/step - accuracy: 0.5240 - loss: 1.3395